# TRIVIATHON

## Team Member:
### Adil Hamid Malla,Sumeet Singh Arora, Rahul Bhagat



## Introduction and Problem Statement
TBD

## Prior Work

TBD


## Basic Plan & Methodology

TBD

## Data Scrapping Using Pywikibot

TBD

In [ ]:
# User Configuration for pywikibot
family = 'wikipedia'
mylang = 'en'
usernames['wikipedia']['en'] = u'ExampleBot'

## Wiki Parser


TBD

In [ ]:
# Wiki Parser Class
import pywikibot
import wiki2plain
import re
import random
from nltk.stem.porter import *
from nltk.corpus import stopwords

class WikiParser:
    def __init__(self):
        print("Making the Instance of Wiki parser.")
        self.site = pywikibot.Site('en', 'wikipedia')
        self.cache_stem = {}
        self.stemmer = PorterStemmer()
        self.stop_words = set(stopwords.words('english'))
        self.k = 50

    def getEntityTokens(self, wiki_entity):
        site = pywikibot.Site('en', 'wikipedia')
        page = pywikibot.Page(site, wiki_entity)  #here we just crawl for the new entry
        text = page.text
        wiki2plain_instance = wiki2plain.Wiki2Plain(text)  #make the text to plain text
        text = wiki2plain_instance.text
        text = text.lower()  #convert all the text to lower case. Case folding
        current_tokens = filter(None, re.split('\W+', text))  #get the tokens now
        current_tokens = [word for word in current_tokens if not word in self.stop_words]
        token_freq_map = {}
        for token in current_tokens:
            token = self.cacheInStem(token)
            if token not in token_freq_map:
                token_freq_map[token] = 1.0
            else:
                token_freq_map[token] += 1.0
        return token_freq_map

    def getCategoryForEntity(self, wiki_entity):
        site = pywikibot.Site('en', 'wikipedia')
        page = pywikibot.Page(site, wiki_entity)
        cat_values = page.categories()
        cat_list = list(cat_values)
        cat_names = []
        for cat in cat_list:
            if not cat.isHiddenCategory():
                cat_names.append(cat.title())
        return cat_names



    def getEntityforCategory(self, category):
        site = pywikibot.Site('en', 'wikipedia')
        catdata = pywikibot.Category(site, title=category)
        entities = catdata.articles()
        return self.getRefinedEntity(entities)

    def cacheInStem(self, token):
        if token not in self.cache_stem:
            self.cache_stem[token] = self.stemmer.stem(token)
        return self.cache_stem[token]

    def getRefinedEntity(self, entities):
        refinedEntity = []
        list_entities = list(entities)
        if len(list_entities) <= self.k:
            for entity in entities:
                refinedEntity.append(entity.title())
            return refinedEntity
        else:
            range_entity = range(0, len(list_entities))
            list_sample = random.sample(range_entity, self.k)
            for i in range(0, self.k):
                refinedEntity.append(list_entities[list_sample[i]].title())
            return refinedEntity


In [ ]:
# Wiki2PlainText

# from http://stackoverflow.com/questions/4460921/extract-the-first-paragraph-from-a-wikipedia-article-python

import re

class Wiki2Plain:
    def __init__(self, wiki):
        self.wiki = wiki

        self.text = wiki
        self.text = self.unhtml(self.text)
        self.text = self.unwiki(self.text)
        self.text = self.punctuate(self.text)

    def __str__(self):
        return self.text

    def unwiki(self, wiki):
        """
        Remove wiki markup from the text.
        """
        wiki = re.sub(r'(?i)\{\{IPA(\-[^\|\{\}]+)*?\|([^\|\{\}]+)(\|[^\{\}]+)*?\}\}', lambda m: m.group(2), wiki)
        wiki = re.sub(r'(?i)\{\{Lang(\-[^\|\{\}]+)*?\|([^\|\{\}]+)(\|[^\{\}]+)*?\}\}', lambda m: m.group(2), wiki)
        wiki = re.sub(r'\{\{[^\{\}]+\}\}', '', wiki)
        wiki = re.sub(r'(?m)\{\{[^\{\}]+\}\}', '', wiki)
        wiki = re.sub(r'(?m)\{\|[^\{\}]*?\|\}', '', wiki)
        wiki = re.sub(r'(?i)\[\[Category:[^\[\]]*?\]\]', '', wiki)
        wiki = re.sub(r'(?i)\[\[Image:[^\[\]]*?\]\]', '', wiki)
        wiki = re.sub(r'(?i)\[\[File:[^\[\]]*?\]\]', '', wiki)
        wiki = re.sub(r'\[\[[^\[\]]*?\|([^\[\]]*?)\]\]', lambda m: m.group(1), wiki)
        wiki = re.sub(r'\[\[([^\[\]]+?)\]\]', lambda m: m.group(1), wiki)
        wiki = re.sub(r'\[\[([^\[\]]+?)\]\]', '', wiki)
        wiki = re.sub(r'(?i)File:[^\[\]]*?', '', wiki)
        wiki = re.sub(r'\[[^\[\]]*? ([^\[\]]*?)\]', lambda m: m.group(1), wiki)
        wiki = re.sub(r"''+", '', wiki)
        wiki = re.sub(r'(?m)^\*$', '', wiki)

        return wiki

    def unhtml(self, html):
        """
        Remove HTML from the text.
        """
        html = re.sub(r'(?i)&nbsp;', ' ', html)
        html = re.sub(r'(?i)<br[ \\]*?>', '\n', html)
        html = re.sub(r'(?m)<!--.*?--\s*>', '', html)
        html = re.sub(r'(?i)<ref[^>]*>[^>]*<\/ ?ref>', '', html)
        html = re.sub(r'(?m)<.*?>', '', html)
        html = re.sub(r'(?i)&amp;', '&', html)

        return html

    def punctuate(self, text):
        """
        Convert every text part into well-formed one-space
        separate paragraph.
        """
        text = re.sub(r'\r\n|\n|\r', '\n', text)
        text = re.sub(r'\n\n+', '\n\n', text)

        parts = text.split('\n\n')
        partsParsed = []

        for part in parts:
            part = part.strip()

            if len(part) == 0:
                continue

            partsParsed.append(part)

        return '\n\n'.join(partsParsed)

    def image(self):
        """
        Retrieve the first image in the document.
        """
        # match = re.search(r'(?i)\|?\s*(image|img|image_flag)\s*=\s*(<!--.*-->)?\s*([^\\/:*?<>"|%]+\.[^\\/:*?<>"|%]{3,4})', self.wiki)
        match = re.search(r'(?i)([^\\/:*?<>"|% =]+)\.(gif|jpg|jpeg|png|bmp)', self.wiki)

        if match:
            return '%s.%s' % match.groups()

        return None


## Trivia Metric Calculator

In [ ]:
# Wiki-Trivia Metric Calculator
# Since by default the encoding scheme is based on the operating system, we will enforce the encoding scheme to be utf-8

import sys

reload(sys)
sys.setdefaultencoding('utf8')

import gensim
import heapq
import math
import Util as util
import pdb

class WikiTriviaMetricCalculator:
    def __init__(self):
        print "Inside the Initialization of the class: WikiTriviaExtractor"
        self.genism_model_filename = "GoogleNews-vectors-negative300.bin"
        self.model = gensim.models.KeyedVectors.load_word2vec_format(self.genism_model_filename, binary=True)
        self.global_idf = util.getglobalfreqdict("plainIdfIndex.txt")
        self.k_val = 10
        self.doc_size = 10000.0  #document size for the idf
        self.rare_term_freq = 10  #used for ignoring rarely occuring terms.
        print "Initialization Done"

    def GetModel(self):
        if self.model:
            return
        print 'Generating the Model'
        self.model = gensim.models.KeyedVectors.load_word2vec_format(self.genism_model_filename, binary=True)
        print 'Model Generated'

    # Get the top k tf idf tokens from the token freq map
    def getTopKTFIDFforEntity(self, token_frequency):
        entity_result = {}
        for token in token_frequency:
            if token not in self.model.vocab:
                continue
            tf = 1.0 + math.log10(token_frequency[token])
            global_freq = self.global_idf[token] if token in self.global_idf else 1.5
            if global_freq < self.rare_term_freq: #ignoring very rare terms
                continue
            entity_result[token] = tf * math.log10(self.doc_size/float(global_freq))
        return heapq.nlargest(self.k_val, entity_result, key=entity_result.get)



    def getEntitySimilarity(self,entity1, entity2):
        sim1 = self.getEntitySimilarityHelper(entity1, entity2)
        sim2 = self.getEntitySimilarityHelper(entity2, entity1)
        return ((sim1 + sim2) / 2.0)

    def getEntitySimilarityHelper(self, entity1, entity2):
        sim = 0.0
        if (len(entity1) < self.k_val or len(entity2) < self.k_val):
            return 0.0
        for i in range(0, self.k_val):
            current_max = self.model.similarity(entity1[i], entity2[0])
            for j in range(1, self.k_val):
                current_val = self.model.similarity(entity1[i], entity2[j])
                if current_val > current_max:
                    current_max = current_val
            sim += (self.k_val - i) * (current_max)
        sim = sim / ((self.k_val+1.0) * (float(self.k_val)))
        sim = sim * 2.0
        return sim

In [ ]:
# Algorithm Wrapper
import wiki_parser
import wiki_trivia_metric_calculator
import Util as util
import pdb
import os
import operator

#globals
category_entity_cache_dir = "catentcache/"
output_cache_dir = "outputCache/"
surprise_weight = 1.1

def triviaAlgorithm(search_entity):
    entity = util.searchWiki(search_entity)
    print "Entity Found" + entity
    # Check for the Output Cache
    full_path = output_cache_dir + entity + ".txt"
    if not os.path.exists(output_cache_dir):
        os.makedirs(output_cache_dir)

    answer_mat = {}
    if os.path.isfile(full_path):
        open_output_file = open(full_path, "r")
        for line in open_output_file:
            trivia, score = line.split(":")
            answer_mat[trivia] = score
        open_output_file.close()
        return answer_mat

    wiki_parser_instance = wiki_parser.WikiParser()
    wiki_trivia_metric_calculator_instance = wiki_trivia_metric_calculator.WikiTriviaMetricCalculator()
    # If the cache doesn't exist- Make the new one for the said entity
    entity_cats = wiki_parser_instance.getCategoryForEntity(entity)
    if not entity_cats:
        return
    if not os.path.exists(category_entity_cache_dir):
        os.makedirs(category_entity_cache_dir)
    for entity_cat in entity_cats:
        surprise_fact = surprise(entity, entity_cat, wiki_parser_instance, wiki_trivia_metric_calculator_instance)
        if surprise_fact:
            answer_mat[entity_cat.split(":")[1]] = surprise_fact
            cohes_score = cohesivness(entity_cat.split(":")[1], wiki_trivia_metric_calculator_instance)
            if cohes_score:
                answer_mat[entity_cat.split(":")[1]] *= cohes_score
            else:
                answer_mat[entity_cat.split(":")[1]] = 0.0
            print "<------------- ----------------->"
            print "Overall score for cat ", entity_cat, " is ", answer_mat[entity_cat.split(":")[1]]
            print "Ending     <------------- ----------------->"
    answer_mat = sorted(answer_mat.items(), key=operator.itemgetter(1), reverse=True)
    target = open(full_path, "w")
    for key in answer_mat:
        target.write(key[0] + ":" + repr(key[1]))
        target.write("\n")
    target.close()
    return answer_mat

def surprise(entity_input, entity_cat, wiki_parser_instance, wiki_trivia_metric_calculator_instance):
    sum = 0.0
    count = 0.0
    entity_input_tokens = wiki_parser_instance.getEntityTokens(entity_input)
    entity_input_top = wiki_trivia_metric_calculator_instance.getTopKTFIDFforEntity(entity_input_tokens)

    path = category_entity_cache_dir + entity_cat.split(":")[1] + "/"
    if os.path.exists(path):
        print "Reading from file "
        outer_list = []
        for(root, dirs, files) in os.walk(path):
            for file in files:
                if file.endswith('.txt'):
                    inner_list = []
                    current_file = open(os.path.join(root, file), "r")
                    for line in current_file:
                        line = line.replace('\n', '')
                        inner_list.append(line)
                    outer_list.append(inner_list)
        size_new = len(outer_list)
        for i in range(0, size_new):
            sum += wiki_trivia_metric_calculator_instance.getEntitySimilarity(entity_input_top, outer_list[i])
            count += 1.0
        answer = sum / count
        print "surprise for ", entity_cat, " is ", (1.0 / answer)
        return (1.0 / answer)

    new_entities = wiki_parser_instance.getEntityforCategory(entity_cat)
    if not new_entities:
        return

    for new_entity in new_entities:
        if new_entity != entity_input:
            new_entity_tokens = wiki_parser_instance.getEntityTokens(new_entity)
            new_entity_tokens_top = wiki_trivia_metric_calculator_instance.getTopKTFIDFforEntity(new_entity_tokens)
            new_entity_top_cache = category_entity_cache_dir + entity_cat.split(':')[1] + "/"
            if not os.path.exists(new_entity_top_cache):
                os.makedirs(new_entity_top_cache)
            cache_file_name = new_entity_top_cache + new_entity + ".txt"
            target = open(cache_file_name, "w")
            for top_token in new_entity_tokens_top:
                target.write(top_token)
                target.write("\n")
            target.close()
            sum += wiki_trivia_metric_calculator_instance.getEntitySimilarity(entity_input_top, new_entity_tokens_top)
            count += 1.0
    answer = sum / count
    print "surprise for " , entity_cat, " is ", (1.0/answer)
    return (1.0 / answer)

def cohesivness(entity_cat, wiki_trivia_metric_calculator_instance):
    sum = 0.0
    count = 0.0
    #answer = sum / count
    path = category_entity_cache_dir + entity_cat + "/"
    outer_list = []
    for(root, dirs, files) in os.walk(path):
        for file in files:
            if file.endswith('.txt'):
                inner_list = []
                current_file = open(os.path.join(root, file), "r")
                for line in current_file:
                    line = line.replace('\n', '')
                    inner_list.append(line)
                outer_list.append(inner_list)
    size_new = len(outer_list)
    for i in range(0, size_new):
        for j in range(i+1, size_new):
                sum += wiki_trivia_metric_calculator_instance.getEntitySimilarity(outer_list[i], outer_list[j])
                count += 1.0
    if count == 0.0:
        return 0.0
    answer = sum / count
    print "Cohes is for cat ", entity_cat, " is ", answer
    return answer



## Main calling Function Entry Points ( This heading will be changed)

In [ ]:
# Main Algorithm Calling Function
import algorithm_wrapper
import wikipedia as wiki
import pdb
if __name__ == "__main__":
    while True:
        input_entity = raw_input()
        if input_entity == "1":
            break
        print algorithm_wrapper.triviaAlgorithm(input_entity)

## Evaluation 

TBD

## Conclusion and Future Work

TBD